In [290]:
%%writefile src/connectAPI.js
const fetch = require('node-fetch')

const KEY = "697456674f6c6a6d313034674d6f6e54"
const TYPE = "json"
const SERVICE = "ListProgramByPublicSportsFacilitiesService"
let START_INDEX = 1
let END_INDEX = 2
let SUBJECT_NAME
let DATA
const url =`http://openapi.seoul.go.kr:8088/${KEY}/${TYPE}/${SERVICE}/${START_INDEX}/${END_INDEX}/`

//console.log(url)

fetch(url)// URL에 대한 GET 요청을 만듬
  .then(response => {
    if (response.ok) {
      return response.json();
    } else {
      throw new Error(`Failed to fetch JSON data from the URL: ${response.status}`);
    }
  })
  .then(jsonData => {
    DATA = jsonData.ListProgramByPublicSportsFacilitiesService
  })
  .catch(error => {
    console.error("Error fetching JSON data:", error);
  });

console.log(DATA)

Writing src/connectAPI.js


In [291]:
%%writefile src/showAPI.js
const fetch = require('node-fetch')

const KEY = "697456674f6c6a6d313034674d6f6e54"
const TYPE = "json"
const SERVICE = "ListProgramByPublicSportsFacilitiesService"
let START_INDEX = 1
let END_INDEX = 1000
let SUBJECT_NAME

async function fetchData(START, END) {
  try {
    const url =`http://openapi.seoul.go.kr:8088/${KEY}/${TYPE}/${SERVICE}/${START}/${END}/`
    const response = await fetch(url);// URL에 대한 GET 요청을 만듬
    if (!response.ok) {
      throw new Error(`Failed to fetch JSON data from the URL: ${response.status}`);// api 값을 받아왔는지 확인
    }
    const jsonData = await response.json();// json으로 값을 반환
    DATA = jsonData.ListProgramByPublicSportsFacilitiesService;
    
    let subjectCount = {}

    for(let i=0; i<DATA.row.length; i++){// 데이터에 따라 값을 json에 넣어줌
      const subjectData = DATA.row[i].SUBJECT_NAME// row별 데이터 접근
      if(subjectData in subjectCount ){
        subjectCount[subjectData] += 1// 데이터가 있으면 value 증가
      }else{
          subjectCount[subjectData] = 1// 데이터가 없으면 새롭게 추가
      }
    }
    console.log(subjectCount)
  } catch (error) {
    console.error("Error fetching JSON data:", error);
  }
}
async function fetchAllData() {
  for(let j = 0; j < 14; j++) {
    await fetchData((1 + 1000 * j), (j + 1) * 1000);
  }
}

fetchAllData();

Writing src/showAPI.js


In [292]:
%%writefile src/mkdirData.js
const fetch = require('node-fetch');
const fs = require('fs');

const KEY = "697456674f6c6a6d313034674d6f6e54";
const TYPE = "json";
const SERVICE = "ListProgramByPublicSportsFacilitiesService";
let START_INDEX = 1;
let END_INDEX = 1000;
let SUBJECT_NAME;

async function fetchData(START, END) {
  try {
    const url =`http://openapi.seoul.go.kr:8088/${KEY}/${TYPE}/${SERVICE}/${START}/${END}/`;
    const response = await fetch(url);
    if (!response.ok) {
      throw new Error(`Failed to fetch JSON data from the URL: ${response.status}`);
    }
    const jsonData = await response.json();
    const DATA = jsonData.ListProgramByPublicSportsFacilitiesService;
    
    let subjectCount = {};

    for(let i = 0; i < DATA.row.length; i++){
      const subjectData = DATA.row[i].SUBJECT_NAME;
      if(subjectData in subjectCount ){
        subjectCount[subjectData] += 1;
      } else {
        subjectCount[subjectData] = 1;
      }
    }
    
    return subjectCount;
  } catch (error) {
    console.error("Error fetching JSON data:", error);
    return {};
  }
}

async function fetchAllData() {
  let allData = {};
  //for(let j = 0; j < 14; j++) {
  //  const data = await fetchData((1 + 1000 * j), (j + 1) * 1000);
  //  allData = { ...allData, ...data };
  //}
  const data = await fetchData(12001, 13000);
  allData = { ...allData, ...data };
  // 파일에 데이터 쓰기
  fs.writeFileSync('output13.json', JSON.stringify(allData, null, 2));
}

fetchAllData();


Writing src/mkdirData.js


In [272]:
! node src/urlCode.js 

(node:5316) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


In [293]:
%%writefile src/combineData.js
const fs = require('fs');

// 모든 JSON 파일을 읽어와서 데이터를 통합할 변수 초기화
let combinedData = {};

// JSON 파일을 읽어와서 데이터를 통합
for (let i = 1; i <= 14; i++) {
  // 파일 이름 설정 (예: input1.json, input2.json, ...)
  const fileName = `data/output${i}.json`;

  // 파일에서 JSON 데이터 읽기
  const jsonData = fs.readFileSync(fileName, 'utf8');

  // JSON 데이터 파싱
  const data = JSON.parse(jsonData);

  // 데이터 통합
  for (const key in data) {
    if (combinedData.hasOwnProperty(key)) {
      combinedData[key] += data[key];
    } else {
      combinedData[key] = data[key];
    }
  }
}

// 통합된 데이터를 파일에 쓰기
fs.writeFileSync('data/combinedOutput.json', JSON.stringify(combinedData, null, 2));

console.log("Data has been combined and saved to combinedOutput.json");

Writing src/combineData.js


In [1]:
! node src/combineData.js 

Data has been combined and saved to combinedOutput.json
